In [4]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [5]:
Q.<rad2_minus_i> = NumberField((sqrt(2)-I).minpoly('xx'))

In [6]:
ii = ((rad2_minus_i)^3+rad2_minus_i)/(-6)

In [8]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [9]:
# Define cubic SE1'', find and classify the lines and compute the 45 tritangent planes

In [62]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1_2 = general_cubic.subs({b:-(c*c+e*f)/c}).subs({e:(c*(2*ii-1)*(5*c-(4*ii-3)*f))/(5*(c-f)), d:((2*ii+1)*(5*c^2-5*f^2-(4*ii+8)*c*f))/(5*(f-c))}).subs({c:-rad2_minus_i, f:3})

In [64]:
# Find simmetries

In [65]:
%time adm_SE1_2 = SE1_2.find_admissible_projectivities()
len(adm_SE1_2)

CPU times: user 3.02 s, sys: 229 ms, total: 3.25 s
Wall time: 16.5 s


576

In [66]:
%time simm_SE1_2 = SE1_2.find_simmetries(adm_SE1_2)
len(simm_SE1_2)

CPU times: user 2.85 s, sys: 221 ms, total: 3.07 s
Wall time: 3.47 s


8

In [67]:
# Study how the simmetries permute the Eckardt points

In [68]:
Eck_perms = [Permutation(SE1_2.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE1_2]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(1, '1')

In [69]:
### geometric interpretation of the group of symmetries

In [70]:
%%time
## Action of the stabilizer of SE1'' on the 27 lines:
Ga = perm_group_lines(SE1_2, simm_SE1_2)

## Action of the stabilizer of SE1'' on the 45 tritangent planes:
Gb = perm_group_planes(SE1_2, simm_SE1_2)

CPU times: user 5.02 s, sys: 24 ms, total: 5.04 s
Wall time: 5.05 s


In [71]:
print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
C8

Structure of the group Gb:
C8


In [72]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [48]:
SE1_1 = general_cubic.subs({b:-(c*c+e*f)/c}).subs({e:(c*(2*ii-1)*(5*c-(4*ii-3)*f))/(5*(c-f)), d:((2*ii+1)*(5*c^2-5*f^2-(4*ii+8)*c*f))/(5*(f-c))})

In [52]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 6.35 s, sys: 250 ms, total: 6.6 s
Wall time: 31.3 s


576

In [53]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 7.12 s, sys: 253 ms, total: 7.37 s
Wall time: 11.4 s


4

In [54]:
%time conds = SE1_1.find_conditions_for_subfamilies(adm_SE1_1, simm_SE1_1)

CPU times: user 6.38 s, sys: 261 ms, total: 6.64 s
Wall time: 40.9 s


In [55]:
len(conds)

24

In [56]:
prim_decs = [el[0] for el in conds]
ideals = list(set([ideal for prim_dec in prim_decs for ideal in prim_dec]))
sosts = [solve_linear_system(ideal.gens(), [c], [f]) for ideal in ideals]

In [57]:
sosts

[((-7*rad2_minus_i^3 + 15*rad2_minus_i^2 + 11*rad2_minus_i - 9)*f, 102*f),
 ((-rad2_minus_i^3 - 15*rad2_minus_i^2 - 19*rad2_minus_i + 21)*f, 102*f),
 ((-rad2_minus_i^3 - rad2_minus_i^2 + rad2_minus_i + 3)*f, 6*f),
 ((-rad2_minus_i)*f, 3*f),
 ((-rad2_minus_i^3 + 2*rad2_minus_i)*f, 9*f),
 ((-rad2_minus_i^3 + 3*rad2_minus_i^2 - 7*rad2_minus_i + 3)*f, 18*f),
 ((-rad2_minus_i^3 + 15*rad2_minus_i^2 - 19*rad2_minus_i - 21)*f, 102*f),
 ((-7*rad2_minus_i^3 - 15*rad2_minus_i^2 + 11*rad2_minus_i + 9)*f, 102*f),
 ((rad2_minus_i^3 + 3*rad2_minus_i^2 + rad2_minus_i - 3)*f, 6*f),
 ((rad2_minus_i^3 - 3*rad2_minus_i^2 + rad2_minus_i + 3)*f, 6*f),
 ((-rad2_minus_i^3 + rad2_minus_i^2 + rad2_minus_i - 3)*f, 6*f),
 ((-rad2_minus_i^3 - 3*rad2_minus_i^2 - 7*rad2_minus_i - 3)*f, 18*f)]

In [59]:
cubics = [SE1_1.subs({c:sost[0], f:sost[1]}) for sost in sosts]
main_cubic_index = sosts.index(((-rad2_minus_i)*f, 3*f)) #we can choose any cubic
main_cubic = cubics[main_cubic_index]

In [73]:
L_sets = [main_cubic.L_set_base]
for L_set in [get_permuted_L_set(perm) for perm in main_cubic.find_admissible_permutations()]:
    if L_set not in L_sets:
        L_sets.append(L_set)

In [74]:
%%time
for cubic in cubics[0:main_cubic_index]+cubics[main_cubic_index+1:]:
    L_set = main_cubic.find_projective_equivalence(cubic, L_sets)
    print(L_set)
    proj = cubic.find_projectivity(cubic.L_set_base, L_set)[0]
    print(main_cubic == cubic.subs(change_coordinates(proj)))

('E1', 'F14', 'F36', 'F15', 'F56')
True
('E1', 'F14', 'F35', 'F16', 'E4')
True
('E1', 'F14', 'F35', 'F12', 'E4')
True
('E1', 'G4', 'E6', 'G5', 'E5')
True
('E1', 'F14', 'F36', 'G3', 'F56')
True
('F14', 'G4', 'E2', 'F26', 'E3')
True
('F14', 'G4', 'F45', 'F23', 'E6')
True
('E1', 'G4', 'E2', 'F12', 'E3')
True
('E1', 'G4', 'F46', 'G6', 'F45')
True
('F14', 'G4', 'E5', 'G1', 'F46')
True
('F14', 'G4', 'E2', 'F25', 'E3')
True
CPU times: user 3min 50s, sys: 480 ms, total: 3min 51s
Wall time: 3min 51s


In [75]:
SE1_2 = main_cubic

In [41]:
%time adm_SE1_2 = SE1_2.find_admissible_projectivities()
len(adm_SE1_2)

CPU times: user 2.76 s, sys: 193 ms, total: 2.95 s
Wall time: 15.8 s


576

In [42]:
%time simm_SE1_2 = SE1_2.find_simmetries(adm_SE1_2)
len(simm_SE1_2)

CPU times: user 2.6 s, sys: 204 ms, total: 2.81 s
Wall time: 3.13 s


8

In [43]:
lines_perms = [Permutation(SE1_2.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE1_2]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(8, 'C8')

In [44]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('G4', 'E1', 'G2', 'E6', 'F15', 'G3')]